In [1]:
import settings
import helpers
import glob
import pandas
import ntpath
import numpy
import cv2
import os

CUBE_IMGTYPE_SRC = "_i"

Computer:  DESKTOP-F8TV69I


# 将从ndsb数据集或者luna数据集里检测到的结节显示出来

In [9]:
if True:
    make_annotation_images_luna16()
    
if False:
    make_annotation_images_ndsb3()


0    	1.3.6.1.4.1.14519.5.2.1.6279.6001.980362852713685276785310240144	5       
Counter: 1
1    	1.3.6.1.4.1.14519.5.2.1.6279.6001.986011151772797848993829243183	1       
Counter: 2
2    	1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950022352718462251777	23      
Counter: 3
3    	1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084993415992563148335	14      
Counter: 4


In [1]:
def make_annotation_images_ndsb3():
    
    src_dir = settings.NDSB3_NODULE_DETECTION_DIR_Model2 + "predictions10_luna16_fs/"
    
    dst_dir = settings.BASE_DIR_SSD + "nodule_check/"
    
    if not os.path.exists(dst_dir):
        os.mkdir(dst_dir)

    for file_path in glob.glob(dst_dir + "*.*"):
        os.remove(file_path)
    
    count=1
    
    #逐一遍历所有病人的positive label文件
    for patient_index, csv_file in enumerate(glob.glob(src_dir + "*.csv")):
        
        patient_id = ntpath.basename(csv_file).replace(".csv", "") 
        df_annos = pandas.read_csv(csv_file)
        
        #如果没有label信息，则跳过
        if len(df_annos) == 0:
            continue
        
        #读取该病人重采样后的图片
        images = helpers.load_patient_images(patient_id, settings.NDSB3_EXTRACTED_IMAGE_DIR, "*" + CUBE_IMGTYPE_SRC + ".png")
        
        #逐一遍历该病人的positive label
        for index, row in df_annos.iterrows():
            
            #读取nodule信息
            coord_x = int(row["coord_x"] * images.shape[2])
            coord_y = int(row["coord_y"] * images.shape[1])
            coord_z = int(row["coord_z"] * images.shape[0])
            anno_index = row["anno_index"]
           
       
            #以nodule的点为中心，创建一个64*64*64的cube
            cube_img = get_cube_from_img(images, coord_x, coord_y, coord_z, 64)
        
            if cube_img.sum() < 5:
                print(" ***** Skipping ", coord_x, coord_y, coord_z)
                continue

            if cube_img.mean() < 10:
                print(" ***** Suspicious ", coord_x, coord_y, coord_z)

            if cube_img.shape != (64, 64, 64):
                print(" ***** incorrect shape !!! ", str(anno_index), " - ",(coord_x, coord_y, coord_z))
                continue
            
            #将该cube数据以8*8个，64*64的图片形式保存
            save_cube_img(dst_dir + patient_id + "_" + str(anno_index) + "_"  + "_1_pos.png", cube_img, 8, 8)
        
        #输入各病人的cube数量
        helpers.print_tabbed([patient_index, patient_id, len(df_annos)], [5, 64, 8])
           
        print("Counter:",count)
        count+=1

In [2]:
def make_annotation_images_luna16():
    
    src_dir = settings.LUNA16_NODULE_DETECTION_DIR_RNN + "predictions10_luna16_fs/"
    
    dst_dir = settings.BASE_DIR_SSD + "nodule_check/"
    
    if not os.path.exists(dst_dir):
        os.mkdir(dst_dir)

    for file_path in glob.glob(dst_dir + "*.*"):
        os.remove(file_path)
    
    count=1
    
    #逐一遍历所有病人的positive label文件
    for patient_index, csv_file in enumerate(glob.glob(src_dir + "*.csv")):
        
        patient_id = ntpath.basename(csv_file).replace(".csv", "") 
        df_annos = pandas.read_csv(csv_file)
        
        #如果没有label信息，则跳过
        if len(df_annos) == 0:
            continue
        
        #读取该病人重采样后的图片
        images = helpers.load_patient_images(patient_id, settings.LUNA16_EXTRACTED_IMAGE_DIR, "*" + CUBE_IMGTYPE_SRC + ".png")
        
        #逐一check nodule
        for index, row in df_annos.iterrows():
            
            #读取nodule信息
            coord_x = int(row["coord_x"] * images.shape[2])
            coord_y = int(row["coord_y"] * images.shape[1])
            coord_z = int(row["coord_z"] * images.shape[0])
            anno_index = row["anno_index"]
           
            #以nodule的点为中心，创建一个64*64*64的cube
            cube_img = get_cube_from_img(images, coord_x, coord_y, coord_z, 64)
        
            if cube_img.sum() < 5:
                print(" ***** Skipping ", coord_x, coord_y, coord_z)
                continue

            if cube_img.mean() < 10:
                print(" ***** Suspicious ", coord_x, coord_y, coord_z)

            if cube_img.shape != (64, 64, 64):
                print(" ***** incorrect shape !!! ", str(anno_index), " - ",(coord_x, coord_y, coord_z))
                continue
            
            #将该cube数据以8*8个，64*64的图片形式保存
            save_cube_img(dst_dir + patient_id + "_" + str(anno_index) + "_"  + "_1_pos.png", cube_img, 8, 8)
        
        #输入各病人的cube数量
        helpers.print_tabbed([patient_index, patient_id, len(df_annos)], [5, 64, 8])
           
        print("Counter:",count)
        count+=1

In [3]:
def get_cube_from_img(img3d, center_x, center_y, center_z, block_size):
    start_x = max(center_x - block_size / 2, 0)
    if start_x + block_size > img3d.shape[2]:
        start_x = img3d.shape[2] - block_size

    start_y = max(center_y - block_size / 2, 0)
    start_z = max(center_z - block_size / 2, 0)
    if start_z + block_size > img3d.shape[0]:
        start_z = img3d.shape[0] - block_size
    start_z = int(start_z)
    start_y = int(start_y)
    start_x = int(start_x)
    res = img3d[start_z:start_z + block_size, start_y:start_y + block_size, start_x:start_x + block_size]
    return res

In [4]:
def save_cube_img(target_path, cube_img, rows, cols):
    #assert rows * cols == cube_img.shape[0]
    img_height = cube_img.shape[1]
    img_width = cube_img.shape[1]
    res_img = numpy.zeros((rows * img_height, cols * img_width), dtype=numpy.uint8)

    for row in range(rows):
        for col in range(cols):
            target_y = row * img_height
            target_x = col * img_width
            res_img[target_y:target_y + img_height, target_x:target_x + img_width] = cube_img[row * cols + col]

    cv2.imwrite(target_path, res_img)